# PDE constrained Topology Optimization

We are again interested in minimizing a shape function of the type
\begin{align*}
     \mathcal J(\Omega) = \int_{\mathsf{D}} (u-u_d)^2 \; dx
\end{align*}
subject to $u:\mathsf{D} \to \mathbb{R}$ solves
$$ \int_{\mathsf{D}} \beta_\Omega\nabla u\cdot \nabla \varphi\;dx = \int_{\mathsf{D}} f_\Omega\varphi \quad \text{ for all } \varphi \in H^1_0(\mathsf{D}), $$
where
<ul>
    <li> $\beta_\Omega = \beta_1\chi_\Omega + \beta_2 \chi_{\mathsf{D}\setminus \Omega},\qquad $  $\beta_1,\beta_2>0$, </li> 
    <li> $f_\Omega = f_1\chi_\Omega + f_2 \chi_{\mathsf{D}\setminus \Omega}, \qquad $  $f_1,f_2\in \mathbb{R}$, </li>     
    <li> $u_d:\mathsf{D} \to \mathbb{R}$.  </li>
</ul>

In [ ]:
from netgen.geom2d import SplineGeometry  # SplieGeometry to define a 2D mesh

from ngsolve import * # import everything from the ngsolve module 
from ngsolve.webgui import Draw

import numpy as np

from interpolations import InterpolateLevelSetToElems # function which interpolates a levelset function


myMAXH = 0.05
EPS = myMAXH * 1e-6      #variable defining when a point is on the interface and when not

We begin with defining the domain $\mathsf{D}$.  For this we define a rectangular mesh with side lenght $R$ using the NGSolve function SplineGeometry() with maximum triangle size maxh. The boundary name of $\partial\mathsf{D}$ is "rectangle".

In [ ]:
geo = SplineGeometry()   

R = 2

## add a rectangle
geo.AddRectangle(p1=(-R,-R),
                 p2=(R,R),
                 bc="rectangle",
                 leftdomain=1,
                 rightdomain=0)


geo.SetMaterial (1, "outer") # give the domain the name "outer"

mesh = Mesh(geo.GenerateMesh(maxh=myMAXH)) # generate ngsolve mesh
Draw(mesh)

<ul>
    <li> fes_state,fes_adj - $H^1$ conforming finite element space of order 1 </li>
    <li> pwc - $L^2$ subspace of functions constant on each element of mesh </li>
    <li> $u,v$ are our trial and test functions </li>
    <li> gfu, gfp are Gridfunctions storing the state and adjoint variable, respectively </li>
    <li> gfud stores $u_d$ - the target function </li>
</ul>

In [ ]:
# H1-conforming finite element space

fes_state = H1(mesh, order=1, dirichlet="rectangle")
fes_adj = H1(mesh, order=1, dirichlet="rectangle")
fes_level = H1(mesh, order=1)

pwc = L2(mesh)   #piecewise constant space


## test and trial functions
u, v = fes_state.TnT()

p, q = fes_adj.TnT()

gfu = GridFunction(fes_state)
gfp = GridFunction(fes_adj)

gfud = GridFunction(fes_state)


We represent the design $\Omega \subset \mathsf{D}$ by means of a level set function $\psi: \mathsf{D} \rightarrow \mathbb R$ in the following way:

\begin{align*}
    \psi(x) < 0 &\quad \Longleftrightarrow\quad  x \in \Omega \\
    \psi(x) = 0 &\quad \Longleftrightarrow\quad  x \in \partial \Omega \\
    \psi(x) > 0 &\quad \Longleftrightarrow\quad  x \in \mathsf{D} \setminus \overline \Omega.
\end{align*}
    <li> psi - gridfunction in fes_state defining $\psi$ </li> 
    <li> psides - gridfunction describing the optimal shape </li>
    <li> psinew - dummy function for line search </li>
</ul>


In [ ]:
psi = GridFunction(fes_level)
psi.Set(1) 

psides = GridFunction(fes_level)
psinew = GridFunction(fes_level)

scene_psi = Draw(psi, mesh, "psi")

Next we solve the state equation
\begin{equation}
\int_{\mathsf{D}} \beta_\Omega \nabla u \cdot \nabla \varphi \;dx = \int_{\mathsf{D}} f_\Omega\varphi \;dx \quad \text{ for all } \varphi \in H^1_0(\mathsf{D}).
\end{equation}
<ul>
    <li> beta, f_rhs - piecewise constant gridfunction </li> 
    <li> B - bilinear form defining $\int_{\mathsf{D}} \beta_\Omega\nabla \psi \cdot \nabla \varphi \;dx$ </li>
    <li> B_adj - bilinear form defining $\int_{\mathsf{D}} \beta_\Omega\nabla \psi \cdot \nabla \varphi \;dx$ </li>
    <li> L - right hand side $\int_{\mathsf{D}} f_\Omega \varphi \;dx$ </li>
</ul>

In [ ]:
# constants for f_rhs and beta
f1 = 10
f2 = 1

beta1 = 10
beta2 = 1

# piecewise constant coefficient functions beta and f_rhs

beta = GridFunction(pwc)
beta.Set(beta1)

f_rhs = GridFunction(pwc)
f_rhs.Set(f1)

#Draw(beta, mesh, "beta")
#Draw(f_rhs, mesh, "f_rhs")

# bilinear form for state equation
B = BilinearForm(fes_state)
B += beta*grad(u) * grad(v) * dx

B_adj = BilinearForm(fes_adj)
B_adj += beta*grad(p) * grad(q) * dx

L = LinearForm(fes_state)
L += f_rhs * v *dx

duCost = LinearForm(fes_adj)


# solving

psides.Set(1)
InterpolateLevelSetToElems(psides, beta1, beta2, beta, mesh, EPS)
InterpolateLevelSetToElems(psides, f1, f2, f_rhs, mesh, EPS)

B.Assemble()
L.Assemble()

inv = B.mat.Inverse(fes_state.FreeDofs(), inverse="sparsecholesky") # inverse of bilinear form
gfu.vec.data = inv*L.vec

scene_u = Draw(gfu)

As an optimal shape we define $\Omega_{opt} := \{f<0\}$, where $f$ is the function from the levelset example describing a clover shape. We construct $u_d$ by solving 
\begin{equation}
\int_{\mathsf{D}} \beta_{\Omega_{opt}} \nabla u_d \cdot \nabla \varphi \;dx = \int_{\mathsf{D}} f_{\Omega_{opt}}\varphi \;dx \quad \text{ for all } \varphi \in H^1_0(\mathsf{D}).
\end{equation}

In [ ]:
a = 4.0/5.0
b = 2
f = CoefficientFunction( 0.1*( (sqrt((x - a)**2 + b * y**2) - 1) \
                * (sqrt((x + a)**2 + b * y**2) - 1) \
                * (sqrt(b * x**2 + (y - a)**2) - 1) \
                * (sqrt(b * x**2 + (y + a)**2) - 1) - 0.001) )

psides.Set(f)
InterpolateLevelSetToElems(psides, beta1, beta2, beta, mesh, EPS)
InterpolateLevelSetToElems(psides, f1, f2, f_rhs, mesh, EPS)

B.Assemble()
L.Assemble()
inv = B.mat.Inverse(fes_state.FreeDofs(), inverse="sparsecholesky") # inverse of bilinear form

gfud.vec.data = inv*L.vec

Draw(gfud, mesh, 'gfud')

The function SolvePDE solves the state and adjoint state each time it is called. Note that since the functions 
beta and f_rhs are GridFunctions we do not have to redefine the bilinearform and linearform when beta and f_rhs change.

In [ ]:
def SolvePDE(adjoint=False):
    #Newton(a, gfu, printing = False, maxerr = 3e-9)

    B.Assemble()
    L.Assemble()

    inv_state = B.mat.Inverse(fes_state.FreeDofs(), inverse="sparsecholesky")
    
    # solve state equation
    gfu.vec.data = inv_state*L.vec

    if adjoint == True: 
        # solve adjoint state equatoin
        duCost.Assemble()
        B_adj.Assemble()
        inv_adj = B_adj.mat.Inverse(fes_adj.FreeDofs(), inverse="sparsecholesky")
        gfp.vec.data = -inv_adj * duCost.vec
    scene_u.Redraw()


InterpolateLevelSetToElems(psi, beta1, beta2, beta, mesh, EPS)
InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS)

SolvePDE()

Redraw()


# define the cost function

def Cost(u):
    return (u - gfud)**2*dx

# derivative of cost function

duCost += 2*(gfu-gfud) * q * dx

    
print("initial cost = ", Integrate(Cost(gfu), mesh))

It remains to implement the topological derivative:
In define $NegPos$ and $PosNeg$ as the topological derivative in $\Omega$ respectively $\Omega^c$:
\begin{align*}
    \mathsf{NegPos}(x) :=  -DJ(\Omega,\omega)(x) &= (-1) * \left(   2 \beta_2 \left(\frac{\beta_2-\beta_1}{\beta_1+\beta_2}\right)\nabla u(x)\cdot \nabla p(x) - (f_2-f_1)p(x) \right) \\
    &= 2 \beta_2 \left(\frac{\beta_1-\beta_2}{\beta_1+\beta_2}\right)\nabla u(x)\cdot \nabla p(x) - (f_1-f_2)p(x) \quad \text{ for }x\in \Omega \\
    \mathsf{PosNeg}(x) := DJ(\Omega,\omega)(x) &= 2 \beta_1 \left(\frac{\beta_1-\beta_2}{\beta_1+\beta_2}\right)\nabla u(x)\cdot \nabla p(x) - (f_1-f_2)p(x) \quad \text{ for }x\in \Omega^c
\end{align*}
and the update function (also called generalised topological derivative)
\begin{equation}
g_\Omega(x):= -\chi_\Omega (x)DJ(\Omega,\omega)(x) + (1-\chi_\Omega)*DJ(\Omega,\omega)(x) . 
\end{equation}

In [ ]:
BetaPosNeg = 2 * beta2 * (beta1-beta2)/(beta1+beta2)    # factors in TD in positive part {x: \psi(x)>0} ={x: \beta(x) = \beta_2}
BetaNegPos = 2 * beta1 * (beta1-beta2)/(beta1+beta2)    # factors in TD in positive part {x: \psi(x)>0} ={x: \beta(x) = \beta_2}
FPosNeg = -(f1-f2)

psinew.vec.data = psi.vec

## normalise psi in L2
normPsi = sqrt(Integrate(psi**2*dx, mesh)) 
psi.vec.data = 1/normPsi * psi.vec

kappa = 0.05

# set level set function to data
InterpolateLevelSetToElems(psi, beta1, beta2, beta, mesh, EPS)
InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS)

Redraw()

TD_node = GridFunction(fes_level)
#scene1 = Draw(TD_node, mesh, "TD_node")

TD_pwc = GridFunction(pwc)
#scene2 = Draw(TD_pwc, mesh, "TD_pwc")

TDPosNeg_pwc = GridFunction(pwc)
#Draw(TDPosNeg_pwc, mesh, "TDPosNeg_pwc")

TDNegPos_pwc = GridFunction(pwc)
#Draw(TDNegPos_pwc, mesh, "TDNegPos_pwc")

cutRatio = GridFunction(pwc)
#scene3 = Draw(cutRatio, mesh, "cutRatio")

# solve for current configuration
SolvePDE(adjoint=True)

Finally we can define a loop defining the levelset update:
\begin{align*}
    \psi_{k+1} = (1-s_k) \frac{\psi_k}{|\psi_k|} + s_k \frac{g_{\psi_k}}{|g_{\psi_k}|}
\end{align*}

In [ ]:
scene = Draw(psi)
scene_cr = Draw(cutRatio, mesh, "cutRatio")
scene_TD0 = Draw(TD_pwc, mesh, "TD_pwc")
scene_TD1 = Draw(TD_node, mesh, "TD_node")

In [ ]:
iter_max = 1000
converged = False



xm=0.
ym=0.
psi.Set( (x-xm)**2+(y-ym)**2-0.25**2)
psinew.vec.data= psi.vec
scene.Redraw()
input("press enter to start optimization")

J = Integrate(Cost(gfu),mesh)

with TaskManager():

    for k in range(iter_max):
        print("================ iteration ", k, "===================")

        # copy new levelset data from psinew into psi
        psi.vec.data = psinew.vec
        scene.Redraw()
        
        SolvePDE(adjoint=True)
        
        J_current = Integrate(Cost(gfu),mesh)

        print( Integrate( (gfu-gfud)**2*dx, mesh) )

        print("cost in beginning of iteration", k, ": Cost = ", J_current)
        
        
        # compute the piecewise constant topological derivative in each domain
        TDPosNeg_pwc.Set( BetaPosNeg * grad(gfu) * grad(gfp)  + FPosNeg*gfp )
        TDNegPos_pwc.Set( BetaNegPos * grad(gfu) * grad(gfp)  + FPosNeg*gfp )

        # compute the cut ratio of the interface elements
        InterpolateLevelSetToElems(psi, 1, 0, cutRatio, mesh, EPS)
        
        # compute the combined topological derivative using the cut ratio information
        for j in range(len(TD_pwc.vec)):
            TD_pwc.vec[j] = cutRatio.vec[j] * TDNegPos_pwc.vec[j] + (1-cutRatio.vec[j])*TDPosNeg_pwc.vec[j]
        
        TD_node.Set(TD_pwc)
        
        scene_cr.Redraw()
        scene_TD0.Redraw()
        scene_TD1.Redraw()
        
        normTD = sqrt(Integrate(TD_node**2*dx, mesh)) # L2 norm of TD_node
        TD_node.vec.data = 1/normTD * TD_node.vec # normalised TD_node
        
        normPsi = sqrt(Integrate(psi**2*dx, mesh)) # L2 norm of psi
        psi.vec.data = 1/normPsi * psi.vec  # normalised psi
        
        linesearch = True
       
        for j in range(10): 

            # update the level set function
            psinew.vec.data = (1-kappa)*psi.vec + kappa*TD_node.vec
            
            # update beta and f_rhs
            InterpolateLevelSetToElems(psinew, beta1, beta2, beta, mesh, EPS)
            InterpolateLevelSetToElems(psinew, f1, f2, f_rhs, mesh, EPS)

            # solve PDE without adjoint
            SolvePDE()
            
            Redraw(blocking=True)
 
            Jnew = Integrate(Cost(gfu), mesh)
            
            if Jnew > J_current:
                print("--------------------")
                print("-----line search ---")
                print("--------------------")
                kappa = kappa*0.8
                print("kappa", kappa)
            else:
                break
        
        Redraw(blocking=True)
        print("----------- Jnew in  iteration ", k, " = ", Jnew, " (kappa = ", kappa, ")")
        print('')
        print("iter" + str(k) + ", Jnew = " + str(Jnew) + " (kappa = ", kappa, ")")
        kappa = min(1, kappa*1.2)

        print("end of iter " + str(k))

input("Finished")

After 999 iterations: $J = 
<center>
    <img src="psi_iter999.jpg" width="800" aligh="center"/>
</center>